In [7]:
from hume import HumeBatchClient
from hume.models.config import LanguageConfig, ProsodyConfig


RECORDING = "../recordings"
hume_api_key = "hCcXf3mVNMtVtNvAO9oz60drAywtf4qsHSVArsg2KFij5LvT"

recording_path = f"{RECORDING}/test.mp3"

client = HumeBatchClient(hume_api_key)
filepaths = [
    recording_path
]

# 음성 분석을 위한 설정을 구성합니다. 필요한 설정을 선택하세요.
## 둘 다 했을 때 걸린 시간: 1m 19.1s
# language_config = LanguageConfig()  # 언어 감정 분석
prosody_config = ProsodyConfig()    # 억양, 음조 분석

# 작업을 제출합니다.
job = client.submit_job(None, [prosody_config], files=filepaths)

print(job)
print("Running...")

# 작업이 완료될 때까지 대기하고, 결과를 다운로드합니다.
details = job.await_complete()
job.download_predictions(f"{RECORDING}/test_prosody_predictions.json")
print("Predictions downloaded to predictions.json")

Job(id="169075ce-3bb9-4a7b-87d8-f0c9c1d50ce2")
Running...
Predictions downloaded to predictions.json


In [3]:
import json
import math

RECORDING = "../recordings"
# JSON 파일 경로
project_id = 10
file_path = f"{RECORDING}/test_prosody_predictions.json"
save_file_path = f"{RECORDING}/test_prosody_predictions_softmax.json"

# JSON 파일 열기
with open(file_path, 'r') as file:
    data = json.load(file)
count = 0
prosodic_data = data[0]['results']['predictions'][0]['models']['prosody']['grouped_predictions'][0]['predictions']

# 각 segment에 대해 softmax를 적용하여 "relative_score" 계산 및 추가
for segment in prosodic_data:
    scores = [item["score"] for item in segment['emotions']]
    exp_scores = [math.exp(score) for score in scores]
    sum_exp_scores = sum(exp_scores)
    
    for item, exp_score in zip(segment['emotions'], exp_scores):
        item["relative_score"] = exp_score / sum_exp_scores  

    relative_scores_sum = sum([item["relative_score"] for item in segment['emotions']])
    print("relative_scores: ", relative_scores_sum)
    
# 수정된 데이터를 원래 JSON 파일에 저장
with open(save_file_path, 'w') as file:
    json.dump(data, file, indent=4)


relative_scores:  1.0
relative_scores:  0.9999999999999996
relative_scores:  1.0
relative_scores:  0.9999999999999998
relative_scores:  0.9999999999999998
relative_scores:  1.0
relative_scores:  1.0
relative_scores:  1.0
relative_scores:  0.9999999999999999
relative_scores:  1.0
relative_scores:  0.9999999999999999
relative_scores:  0.9999999999999996
relative_scores:  0.9999999999999997
relative_scores:  1.0000000000000002
relative_scores:  1.0000000000000002
relative_scores:  1.0000000000000002
relative_scores:  1.0
relative_scores:  1.0000000000000002
relative_scores:  0.9999999999999998
relative_scores:  0.9999999999999999
relative_scores:  1.0000000000000002
relative_scores:  0.9999999999999997
relative_scores:  1.0000000000000004
relative_scores:  0.9999999999999994
relative_scores:  1.0000000000000002
relative_scores:  0.9999999999999998
relative_scores:  1.0
relative_scores:  1.0
relative_scores:  0.9999999999999999
relative_scores:  0.9999999999999997
relative_scores:  0.99999

In [16]:
pdata = data[0]['results']['predictions'][0]['models']['prosody']['grouped_predictions'][0]['predictions']


[{'text': 'So welcome to Lecture two of Cs s two three one n.',
  'time': {'begin': 0.27786955, 'end': 2.7389998},
  'confidence': 0.932251,
  'speaker_confidence': None,
  'emotions': [{'name': 'Admiration', 'score': 0.13697195053100586},
   {'name': 'Adoration', 'score': 0.04777386412024498},
   {'name': 'Aesthetic Appreciation', 'score': 0.1116311103105545},
   {'name': 'Amusement', 'score': 0.08085121959447861},
   {'name': 'Anger', 'score': 0.0017994032241404057},
   {'name': 'Anxiety', 'score': 0.002625252353027463},
   {'name': 'Awe', 'score': 0.02320512756705284},
   {'name': 'Awkwardness', 'score': 0.018507426604628563},
   {'name': 'Boredom', 'score': 0.010537137277424335},
   {'name': 'Calmness', 'score': 0.13825035095214844},
   {'name': 'Concentration', 'score': 0.2055852860212326},
   {'name': 'Confusion', 'score': 0.009000740014016628},
   {'name': 'Contemplation', 'score': 0.030096787959337234},
   {'name': 'Contempt', 'score': 0.004864085931330919},
   {'name': 'Conten

In [11]:
import json
import math

RECORDING = "../recordings"
# JSON 파일 경로
project_id = 10
file_path = f"{RECORDING}/{project_id}_prosody_predictions.json"

# JSON 파일 열기
with open(file_path, 'r') as file:
    data = json.load(file)
count = 0
prosodic_data = data[0]['results']['predictions'][0]['models']['prosody']['grouped_predictions'][0]['predictions']

positve_emotion = ['Excitement', 'Enthusiasm', 'Interest', 'Amusement', 'Joy']
negative_emotion = ['Calmness', 'Boredom', 'Tiredness']

score_sum = []
positive_relative_score_sum = []
negative_relative_score_sum = []

for segment in prosodic_data:
    scores = [item["score"] for item in segment['emotions']]
    score_sum.append(sum(scores))
    positive_relative_score_sum.append(sum([item["relative_score"] for item in segment['emotions'] if item["name"] in positve_emotion]))
    negative_relative_score_sum.append(sum([item["relative_score"] for item in segment['emotions'] if item["name"] in negative_emotion]))

score_avg = sum(score_sum)/len(score_sum)
positive_relative_score_avg = sum(positive_relative_score_sum)/len(positive_relative_score_sum)
negative_relative_score_avg = sum(negative_relative_score_sum)/len(negative_relative_score_sum)

important_parts = []
for segment in prosodic_data:
    if sum([item["score"] for item in segment['emotions']]) > score_avg and sum([item["relative_score"] for item in segment['emotions'] if item["name"] in positve_emotion]) > positive_relative_score_avg and sum([item["relative_score"] for item in segment['emotions'] if item["name"] in negative_emotion]) < negative_relative_score_avg:
        important_parts.append(segment['time'])

print(len(important_parts))

45


In [16]:
import json
import numpy as np

RECORDING = "../recordings"
# JSON 파일 경로
project_id = 10
file_path = f"{RECORDING}/{project_id}_prosody_predictions.json"

# JSON 파일 열기
with open(file_path, 'r') as file:
    data = json.load(file)

positve_emotion = ['Excitement', 'Enthusiasm', 'Interest', 'Amusement', 'Joy']
negative_emotion = ['Calmness', 'Boredom', 'Tiredness']

prosodic_data = data[0]['results']['predictions'][0]['models']['prosody']['grouped_predictions'][0]['predictions']

scores = []
positive_relative_scores = []
negative_relative_scores = []

for segment in prosodic_data:
    segment_scores = [item["score"] for item in segment['emotions']]
    scores.append(sum(segment_scores))
    
    positive_relative_score_sum = sum([item["relative_score"] for item in segment['emotions'] if item["name"] in positve_emotion])
    positive_relative_scores.append(positive_relative_score_sum)
    
    negative_relative_score_sum = sum([item["relative_score"] for item in segment['emotions'] if item["name"] in negative_emotion])
    negative_relative_scores.append(negative_relative_score_sum)

score_st = np.percentile(scores, 90)
positive_relative_score_st = np.percentile(positive_relative_scores, 90)
negative_relative_score_st = np.percentile(negative_relative_scores, 10)

important_parts = []
for segment, score_sum, positive_rel_sum, negative_rel_sum in zip(prosodic_data, scores, positive_relative_scores, negative_relative_scores):
    if score_sum > score_st and positive_rel_sum > positive_relative_score_st and negative_rel_sum < negative_relative_score_st:
        important_parts.append([segment['time']['begin'], segment['time']['end']])

print(important_parts)

[[66.2, 71.0], [350.95477, 355.44687], [413.02567, 417.96667], [541.899, 546.6821], [1347.5665, 1351.7979]]
